<div style="position:relative; height:160px;">
<img src="logoDECATUFS.png" width="500" height="500" style="position:absolute; left:-8px;">
</div>

<div style="text-align:center;font-size:17px; white-space:pre;margin-top:-35px">
        <h3>             Modelos Lineares Generalizados</h3>
        Disciplina ofertada pelo DECAT/UFS
        Código: ESTAT0092
        Nível: Graduação
        Carga horária: 60h
        Período: 2020.2
        Professor responsável e ministrante: Luiz Henrique Dore
        <h3>            Aula 12: testanto a significância global do modelo</h3>
</div>

# Exemplo 2

<p style="text-align: justify; font-size:16px">
Considere o conjunto de dados <b>icu.csv</b>, descrito no exemplo 3 da aula 2. O objetivo é ajustar um modelo de regressão logística, no qual a variável resposta é a variável <b>sta</b> (Sobrevivência) e a variável preditora é a variável <b>age</b> (Idade), e executar o teste da razão de verossimilhanças para avaliar a significância global do modelo.
<p>

### 2.1 Leitura e visualização dos dados

In [1]:
dados<-read.csv("/Dados/Google_Drive_UFS/Aulas/2020_2/MLG/Dados/icu.csv",header=TRUE)
n<-dim(dados)[1] #Número de observações
head(dados) #Visualizando as primeiras observações

,id,sta,age,gender,race,ser,can,crn,inf,cpr,⋯,hra,pre,type,fra,po2,ph,pco,bic,cre,loc
,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,4,Died,87,Female,White,Surgical,No,No,Yes,No,⋯,96,No,Emergency,Yes,<= 60,< 7.25,> 45,>= 18,<= 2.0,Nothing
2,8,Lived,27,Female,White,Medical,No,No,Yes,No,⋯,88,No,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
3,12,Lived,59,Male,White,Medical,No,No,No,No,⋯,80,Yes,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
4,14,Lived,77,Male,White,Surgical,No,No,No,No,⋯,70,No,Elective,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
5,27,Died,76,Female,White,Surgical,No,No,Yes,No,⋯,90,Yes,Emergency,No,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing
6,28,Lived,54,Male,White,Medical,No,No,Yes,No,⋯,103,No,Emergency,Yes,> 60,>= 7.25,<= 45,>= 18,<= 2.0,Nothing


In [2]:
sta<-as.numeric(dados[,"sta"]=="Lived") #0 para Died e 1 para Lived
age<-dados[,3]

### 2.2 Formulação do modelo

<p style="text-align: justify; font-size:15px">
    <ol style="font-size:16px">
        <li>$Sobrevivência\sim Bernoulli(\mu)$;</li>
        <li>$\eta=\beta_0+\beta_1\cdot Idade$;</li>
        <li>$\ln\left(\frac{\mu}{1-\mu}\right)=\eta$.</li>
    </ol>
</p>

### 2.3 Estimação dos parâmetros pelo método escore de Fisher usando a função ``glm``

<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão no modelo completo.</p>

In [3]:
aj<-glm(formula=sta~age,family=binomial(link="logit"),data=dados)
beta<-aj$coefficients

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos coeficientes de regressão do modelo completo.</p>

In [4]:
print(beta)

(Intercept)         age 
 3.05851323 -0.02754261 


<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão do modelo sob $H_0$.</p>

In [5]:
aj0<-glm(formula=sta~1,family=binomial(link="logit"),data=dados)
beta0<-aj0$coefficients

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos coeficientes de regressão do modelo sob $H_0$.</p>

In [6]:
print(beta0)

(Intercept) 
   1.386294 


### 2.4 Executando o teste da razão de verossimilhanças

<p style="text-align: justify; font-size:16px">No caso da distribuição gama, a função de log-verossimilhança é dada por (ver a fórmula 2 no tópico 10 e o exemplo 3 no tópico 12)
    $$l(\boldsymbol{\beta},\phi)=\phi^{-1}\sum\limits_{i=1}^n[\theta_iy_i-b(\theta_i)]+\sum\limits_{i=1}^nc(y_i,\phi),$$
onde $\theta_i=\ln\left(\frac{\mu_i}{1-\mu_i}\right)$, $b(\theta_i)=\ln\left(1+e^{\theta_i}\right)$, $\phi=1$ e $c(y_i,\phi)=0$. Primeiramente, utilizando a função <font face="'Courier New'">predict</font>, calcula-se os valores de $\mu_i$  Utilizando os valores de $\mu_i$, calcula-se os valores de $\theta_i$ e, utilizando os valores de $\theta_i$, calcula-se os valores de $b(\theta_i)$. Por último, substitui-se os valores de $\theta_i$ e $b(\theta_i)$ na fórmula da função de log-verossimilhança.

<p style="text-align: justify;font-size: 16px">Calculando a log-verossimilhança do modelo completo.</p>

In [7]:
y<-sta #Atribuindo os valores observados da resposta ao vetor y
mu<-predict(aj,type="response") #Calculando os valores de mu com base no modelo ajustado
theta<-log(mu/(1-mu)) #Calculando os valores de theta a partir de mu
btheta<-log(1+exp(theta)) #Calculando os valores de b(theta) a partir de theta
l<-sum(theta*y-btheta) #Calculando a log-verossimilhança
l<-as.numeric(l)
print(paste("log-verossimilhança do modelo completo: ",l))

[1] "log-verossimilhança do modelo completo:  -96.1531901918606"


<p style="text-align: justify;font-size: 16px">Calculando a log-verossimilhança do modelo sob $H_0$.</p>

In [8]:
y<-sta #Atribuindo os valores observados da resposta ao vetor y
mu0<-predict(aj0,type="response") #Calculando os valores de mu com base no modelo ajustado
theta0<-log(mu0/(1-mu0)) #Calculando os valores de theta a partir de mu
btheta0<-log(1+exp(theta0)) #Calculando os valores de b(theta) a partir de theta
l0<-sum(theta0*y-btheta0) #Calculando a log-verossimilhança
l0<-as.numeric(l0)
print(paste("log-verossimilhança do modelo completo: ",l0))

[1] "log-verossimilhança do modelo completo:  -100.080484707638"


<p style="text-align: justify;font-size: 16px">Calculando a estatística do teste de da razão de verossimilhanças $T_{RV}=2\left\{l\left(\boldsymbol{\widehat\beta}\right)-l\left(\hat\beta^0_0\right)\right\}$.</p>

In [9]:
tRV<-2*(l-l0)
print(paste("Valor observado da estatística do teste: ",tRV))

[1] "Valor observado da estatística do teste:  7.8545890315539"


<p style="text-align: justify;font-size: 16px">Calculando o p-valor do teste. Sob $H_0$, a estatística $T_{RV}$ segue, aproximadamnete, distribuição $\chi_p^2$, onde $p$ é o número de variáveis preditoras. Logo, o p-valor fica dado por $P(T_{RV}>t_{RV})=1-P(T_{RV}\leq t_{RV})$, onde $T_{RV}\sim\chi_p^2$ e $t_{RV}$ é o valor observado de $T_{RV}$.</p>

In [10]:
p<-length(beta)-1 #O número de variáveis preditoras é o número de coeficientes menos 1 (excluindo o intercepto) 
pvalor<-1-pchisq(tRV,p) 
print(paste("p-valor do teste: ",pvalor))

[1] "p-valor do teste:  0.0050691873742762"


### 2.5 Executando o teste da razão de verossimilhanças usando a função ``lr.test``

<p style="text-align: justify;font-size: 16px">Carregando os pacotes necessários.</p>

In [11]:
library(mdscore) #Contém a função lr.test
#install.packages("mdscore") #Execute esse comando para instalar o pacote mdscore, caso já não o tenha instalado

Loading required package: MASS



<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">lr.test</font> realiza o teste da razão de verossimilhanças para a significância global do modelo. A função recebe dois argumentos:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">fit1</font>: modelo ajustado sob $H_0$. Saída da função <font face="'Courier New'">glm</font> referente ao modelo sob $H_0$.</li>
        <li><font face="'Courier New'">fit2</font>: modelo completo ajustado. Saída da função <font face="'Courier New'">glm</font> referente ao modelo completo.</li>
    </ol>
</p>

In [12]:
lr.test(fit1=aj0,fit2=aj)

$LR
[1] 7.854589

$pvalue
[1] 0.005069187

attr(,"class")
[1] "lrt.test"

<p style="text-align: justify;font-size: 16px">Calculando a exponencial dos coeficientes de regressão.</p>

<p style="text-align: justify; font-size:16px">
O p-valor é menor do que 1%, indicando a rejeição de $H_0$, ao nível de 1% de significância. Ou seja, o resultado do teste indica que há uma associação estatisticamente significativa entre a probabilidade de sobrevivência do paciente e sua idade.
</p>

In [13]:
p

[1] 1